In [2]:
# Names of folders and variables

path_to_data_files = '/Users/yardenc/Dropbox/Cohen_CanaryBoutAnnotation/lb4483/Part 1/SyllableSpects'
file_prefix = 'syllable_spects_'

# imports

import numpy as np
import scipy.io as cpio
import os
import matplotlib.pyplot as plt
import glob
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import h5py


np.random.seed(42)

# Work on all files
os.chdir(path_to_data_files)
matfiles = glob.glob(file_prefix + '*.mat')
print(matfiles)
numfiles = len(matfiles)

for fnum0 in range(46,numfiles):
    # skipped 4 because of memory issues
    # remember to change back the range
    fname0 = matfiles[fnum0]
    
    print('Working on ' + fname0)
    f = h5py.File(fname0)
    num_syls0 = len(f['syllable_spects']['samples'][0])
    
    max_entries = 20000
    
    #data0 = cpio.loadmat(fname0) 
    #num_syls0 = len(data0['syllable_spects'][0][0][1])
    print('We have ' + str(num_syls0)  + ' syllables in this class');

# Find max number of time bins  and complete spectrograms by zero padding
# (keeping the spectrogram centered)
    #max_bins = np.max([a[1] for a in [np.shape(np.array(f[f['syllable_spects']['samples'][0][k]])) for k in range(num_syls0)]])
    max_bins = np.max([a[0] for a in [np.shape(np.array(f[f['syllable_spects']['samples'][0][k]])) for k in range(num_syls0)]])
    #max_bins = np.max([a[1] for a in [np.shape(data0['syllable_spects'][0][0][1][k][0]) for k in range(num_syls0)]])
    currsylidx = 0;
    output_y = np.asarray([]);
    while currsylidx < num_syls0:
        data1 = [];
        print(currsylidx)
        for sylcnt in range(currsylidx,min(num_syls0,currsylidx+max_entries)):
            #[a,numbins] = np.shape(data0['syllable_spects'][0][0][1][sylcnt][0]) #np.array(f[f['syllable_spects']['samples'][0][sylcnt]])) #
            [numbins,a] = np.shape(np.array(f[f['syllable_spects']['samples'][0][sylcnt]]))
            bins_to_add = max_bins - numbins
            bins_before = bins_to_add/2
            bins_after = bins_to_add - bins_before
            #data0['syllable_spects'][0][0][1][sylcnt][0] = np.concatenate(
            #    (np.zeros((513,bins_before)),
            #    data0['syllable_spects'][0][0][1][sylcnt][0],
            #    np.zeros((513,bins_after))),axis=1)
            data1.append(np.concatenate(
                (np.zeros((513,bins_before)),
                np.transpose(f[f['syllable_spects']['samples'][0][sylcnt]]),
                np.zeros((513,bins_after))),axis=1))

        print('All spectrograms were zero padded to equally have ' + str(max_bins) + ' time bins.')        

    # Reshape dataset
        #X0 = [np.reshape(data0['syllable_spects'][0][0][1][k][0],(-1,)) for k in range(num_syls0)]
        #X0 = [np.reshape(f[f['syllable_spects']['samples'][0][k]],(-1,)) for k in range(num_syls0)]
        X0 = [np.reshape(data1[k],(-1,)) for k in range(np.size(data1,0))]

    # Train model
        clf = IsolationForest(contamination=0.03,
                                             random_state=42)
        #LocalOutlierFactor(n_neighbors=20, contamination=0.01)

        y_pred = clf.fit(X0).predict(X0) #fit_predict(X0)
        output_y = np.concatenate((output_y,y_pred),axis=0)
        currsylidx = currsylidx + max_entries
        
# save results
    output_filename = 'output_' + fname0
    cpio.savemat(output_filename,{'output':output_y})
        

/Users/yardenc/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['syllable_spects_1.mat', 'syllable_spects_10.mat', 'syllable_spects_11.mat', 'syllable_spects_12.mat', 'syllable_spects_13.mat', 'syllable_spects_14.mat', 'syllable_spects_15.mat', 'syllable_spects_16.mat', 'syllable_spects_17.mat', 'syllable_spects_18.mat', 'syllable_spects_19.mat', 'syllable_spects_2.mat', 'syllable_spects_20.mat', 'syllable_spects_21.mat', 'syllable_spects_22.mat', 'syllable_spects_23.mat', 'syllable_spects_24.mat', 'syllable_spects_25.mat', 'syllable_spects_26.mat', 'syllable_spects_27.mat', 'syllable_spects_28.mat', 'syllable_spects_29.mat', 'syllable_spects_3.mat', 'syllable_spects_30.mat', 'syllable_spects_31.mat', 'syllable_spects_32.mat', 'syllable_spects_33.mat', 'syllable_spects_34.mat', 'syllable_spects_35.mat', 'syllable_spects_36.mat', 'syllable_spects_37.mat', 'syllable_spects_38.mat', 'syllable_spects_39.mat', 'syllable_spects_4.mat', 'syllable_spects_40.mat', 'syllable_spects_41.mat', 'syllable_spects_42.mat', 'syllable_spects_43.mat', 'syllable_spect

/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


20000
All spectrograms were zero padded to equally have 33 time bins.


/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


...

/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


120000
All spectrograms were zero padded to equally have 33 time bins.


/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


140000
All spectrograms were zero padded to equally have 33 time bins.


/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [1]:
fnum0

NameError: name 'fnum0' is not defined

In [16]:
currsylidx=0
while currsylidx < num_syls0:
    data1 = [];
    print(currsylidx)
    for sylcnt in range(currsylidx,min(num_syls0,currsylidx+max_entries)):
        #[a,numbins] = np.shape(data0['syllable_spects'][0][0][1][sylcnt][0]) #np.array(f[f['syllable_spects']['samples'][0][sylcnt]])) #
        [numbins,a] = np.shape(np.array(f[f['syllable_spects']['samples'][0][sylcnt]]))
        bins_to_add = max_bins - numbins
        bins_before = bins_to_add/2
        bins_after = bins_to_add - bins_before
        #data0['syllable_spects'][0][0][1][sylcnt][0] = np.concatenate(
        #    (np.zeros((513,bins_before)),
        #    data0['syllable_spects'][0][0][1][sylcnt][0],
        #    np.zeros((513,bins_after))),axis=1)
        data1.append(np.concatenate(
            (np.zeros((513,bins_before)),
            np.transpose(f[f['syllable_spects']['samples'][0][sylcnt]]),
            np.zeros((513,bins_after))),axis=1))

    print('All spectrograms were zero padded to equally have ' + str(max_bins) + ' time bins.')        

# Reshape dataset
    #X0 = [np.reshape(data0['syllable_spects'][0][0][1][k][0],(-1,)) for k in range(num_syls0)]
    #X0 = [np.reshape(f[f['syllable_spects']['samples'][0][k]],(-1,)) for k in range(num_syls0)]
    X0 = [np.reshape(data1[k],(-1,)) for k in range(np.size(data1,0))]

# Train model
    clf = IsolationForest(contamination=0.03,
                                         random_state=42)
    #LocalOutlierFactor(n_neighbors=20, contamination=0.01)

    y_pred = clf.fit(X0).predict(X0) #fit_predict(X0)
    output_y = np.concatenate() append(y_pred)
    currsylidx = currsylidx + max_entries


0
All spectrograms were zero padded to equally have 38 time bins.


/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


20000
All spectrograms were zero padded to equally have 38 time bins.


/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


40000
All spectrograms were zero padded to equally have 38 time bins.


...

/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


140000
All spectrograms were zero padded to equally have 38 time bins.


/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


160000
All spectrograms were zero padded to equally have 38 time bins.


/Users/yardenc/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [59]:
f['syllable_spects']['samples'][0]

IOError: Can't read data (Inflate() failed)

In [1]:
path_to_data_files = '/Users/yardenc/Dropbox/Cohen_CanaryBoutAnnotation/lb4483/Part 1/SyllableSpects'
file_prefix = 'syllable_spects_'

# imports

import numpy as np
import scipy.io as cpio
import os
import matplotlib.pyplot as plt
import glob
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import h5py

np.random.seed(42)

# Work on all files
os.chdir(path_to_data_files)
matfiles = glob.glob(file_prefix + '*.mat')
print(matfiles)
numfiles = len(matfiles)

fnum0=4
fname0 = matfiles[fnum0]
    
print('Working on ' + fname0)
f = h5py.File(fname0)
num_syls0 = len(f['syllable_spects']['samples'][0])
data1 = [];
#data0 = cpio.loadmat(fname0) 
#num_syls0 = len(data0['syllable_spects'][0][0][1])
print('We have ' + str(num_syls0)  + ' syllables in this class');

# Find max number of time bins  and complete spectrograms by zero padding
# (keeping the spectrogram centered)
#max_bins = np.max([a[1] for a in [np.shape(np.array(f[f['syllable_spects']['samples'][0][k]])) for k in range(num_syls0)]])
max_bins = np.max([a[0] for a in [np.shape(np.array(f[f['syllable_spects']['samples'][0][k]])) for k in range(num_syls0)]])

/Users/yardenc/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['syllable_spects_1.mat', 'syllable_spects_10.mat', 'syllable_spects_11.mat', 'syllable_spects_12.mat', 'syllable_spects_13.mat', 'syllable_spects_14.mat', 'syllable_spects_15.mat', 'syllable_spects_16.mat', 'syllable_spects_17.mat', 'syllable_spects_18.mat', 'syllable_spects_19.mat', 'syllable_spects_2.mat', 'syllable_spects_20.mat', 'syllable_spects_21.mat', 'syllable_spects_22.mat', 'syllable_spects_23.mat', 'syllable_spects_24.mat', 'syllable_spects_25.mat', 'syllable_spects_26.mat', 'syllable_spects_27.mat', 'syllable_spects_28.mat', 'syllable_spects_29.mat', 'syllable_spects_3.mat', 'syllable_spects_30.mat', 'syllable_spects_31.mat', 'syllable_spects_32.mat', 'syllable_spects_33.mat', 'syllable_spects_34.mat', 'syllable_spects_35.mat', 'syllable_spects_36.mat', 'syllable_spects_37.mat', 'syllable_spects_38.mat', 'syllable_spects_39.mat', 'syllable_spects_4.mat', 'syllable_spects_40.mat', 'syllable_spects_41.mat', 'syllable_spects_42.mat', 'syllable_spects_43.mat', 'syllable_spect

In [3]:
max_bins

38

In [37]:
f[f['syllable_spects']['samples'][0][sylcnt]].dtype

dtype('float32')

In [24]:
data0

{'__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Fri Jun 14 11:50:35 2019',
 '__version__': '1.0',
 'syllable_spects': array([[(array([[1]], dtype=uint8), array([[array([[2.7197635e+00, 3.2908945e+00, 2.9610467e+00, ..., 2.2231159e+00,
         9.4844133e-01, 8.8988906e-01],
        [2.7330649e+00, 3.2697740e+00, 3.1010237e+00, ..., 2.4715652e+00,
         1.5949491e+00, 1.1530445e+00],
        [2.7160082e+00, 3.1893210e+00, 3.2286520e+00, ..., 2.7410052e+00,
         2.2056837e+00, 1.8597271e+00],
        ...,
        [2.0934320e-03, 1.6972626e-02, 1.8703430e-03, ..., 3.5088749e-03,
         1.6438991e-02, 6.8586483e-03],
        [2.0379955e-03, 1.9631032e-02, 3.3678948e-03, ..., 2.8737260e-03,
         1.9259669e-02, 6.7941044e-03],
        [1.8457487e-03, 2.0320119e-02, 3.4984760e-03, ..., 1.5297254e-03,
         2.4142034e-02, 1.3686306e-02]], dtype=float32)],
        [array([[4.5966196e-01, 4.6886551e-01, 1.7817515e+00, ..., 2.3137362e+00,
   

In [25]:
num_syls0 = len(data0['syllable_spects'][0][0][1])

In [26]:
num_syls0

2538